# Stock Etablissement

In [1]:
import polars as pl
import pyarrow.parquet as pq
import sys

# Indique le chemin de ton fichier
filepath = "../Data/raw/StockEtablissement_utf8.parquet" 

print("--- Début de la lecture 'bypass' ---")

try:
    # ÉTAPE 1 : On lit avec PyArrow, qui est robuste.
    print(f"Lecture du fichier via PyArrow : {filepath}")
    table_arrow = pq.read_table(
        filepath,
    )
    
    # ÉTAPE 2 : On passe la data à Polars sans la copier (gain de performance).
    print("Conversion de la table PyArrow en DataFrame Polars...")
    df_eta = pl.from_arrow(table_arrow)
    
    print("--- SUCCÈS ! ---")
    print("Le DataFrame est maintenant dans Polars, prêt pour la transformation.")
    print(df_eta.head())

except Exception as e:
    print(f"--- ERREUR ---", file=sys.stderr)
    print(f"Impossible de lire le fichier, même avec PyArrow : {e}", file=sys.stderr)

--- Début de la lecture 'bypass' ---
Lecture du fichier via PyArrow : ../Data/raw/StockEtablissement_utf8.parquet
Conversion de la table PyArrow en DataFrame Polars...
--- SUCCÈS ! ---
Le DataFrame est maintenant dans Polars, prêt pour la transformation.
shape: (5, 53)
┌───────────┬─────┬────────────┬────────────┬───┬────────────┬────────────┬────────────┬───────────┐
│ siren     ┆ nic ┆ siret      ┆ statutDiff ┆ … ┆ denominati ┆ activitePr ┆ nomenclatu ┆ caractere │
│ ---       ┆ --- ┆ ---        ┆ usionEtabl ┆   ┆ onUsuelleE ┆ incipaleEt ┆ reActivite ┆ Employeur │
│ str       ┆ i64 ┆ str        ┆ issement   ┆   ┆ tablisseme ┆ ablissemen ┆ Principale ┆ Etablisse │
│           ┆     ┆            ┆ ---        ┆   ┆ …          ┆ …          ┆ …          ┆ men…      │
│           ┆     ┆            ┆ str        ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---       │
│           ┆     ┆            ┆            ┆   ┆ str        ┆ str        ┆ str        ┆ str       │
╞═══════════╪═════╪════

In [5]:
df_eta.describe()

statistic,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,dernierNumeroVoieEtablissement,indiceRepetitionDernierNumeroVoieEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,identifiantAdresseEtablissement,coordonneeLambertAbscisseEtablissement,coordonneeLambertOrdonneeEtablissement,complementAdresse2Etablissement,numeroVoie2Etablissement,indiceRepetition2Etablissement,typeVoie2Etablissement,libelleVoie2Etablissement,codePostal2Etablissement,libelleCommune2Etablissement,libelleCommuneEtranger2Etablissement,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
str,str,f64,str,str,str,str,f64,str,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""42151993""",4.2151993e7,"""42151993""","""42151993""","""38722345""","""42151993""",2.48317e6,"""3047049""","""42151993""",4.2151993e7,4.2151993e7,"""11792209""","""34192894""","""6288682""","""4908736""","""4896366""","""37116678""","""41334261""","""41846468""","""41809918""","""312355""","""4896366""","""41809918""","""4896366""","""4896366""","""338770""","""338756""","""33421437""","""31584696""","""31584696""","""4896366""","""4896366""","""4896366""","""4896366""","""4896366""","""4896366""","""0""","""0""","""4896366""","""0""","""4896366""","""4896366""","""0""","""0""","""41633958""","""42151993""","""8067086""","""4868843""","""4853220""","""8569843""","""42130555""","""42130555""","""41981193"""
"""null_count""","""0""",0.0,"""0""","""0""","""3429648""","""0""",3.9668823e7,"""39104944""","""0""",0.0,0.0,"""30359784""","""7959099""","""35863311""","""37243257""","""37255627""","""5035315""","""817732""","""305525""","""342075""","""41839638""","""37255627""","""342075""","""37255627""","""37255627""","""41813223""","""41813237""","""8730556""","""10567297""","""10567297""","""37255627""","""37255627""","""37255627""","""37255627""","""37255627""","""37255627""","""42151993""","""42151993""","""37255627""","""42151993""","""37255627""","""37255627""","""42151993""","""42151993""","""518035""","""0""","""34084907""","""37283150""","""37298773""","""33582150""","""21438""","""21438""","""170800"""
"""mean""",null,136.607893,null,null,"""2008-11-05 10:29:04.515152""",null,2023.0,null,"""2024-09-18 21:41:53.413371""",0.684679,2.226143,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""2013-06-28 12:15:30.213130""",null,null,null,null,null,null,null,null
"""std""",null,1926.657096,null,null,null,null,0.0,null,null,null,1.306049,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""000325175""",1.0,"""00032517500016""","""O""","""0001-01-16""","""00""",2023.0,""" ""","""2024-03-22 15:40:57""",0.0,1.0,""" HAMEAU DE PARUEL""",""" 5""",""" ""","

In [4]:
df_eta.schema

Schema([('siren', String),
        ('nic', Int64),
        ('siret', String),
        ('statutDiffusionEtablissement', String),
        ('dateCreationEtablissement', Date),
        ('trancheEffectifsEtablissement', String),
        ('anneeEffectifsEtablissement', Int64),
        ('activitePrincipaleRegistreMetiersEtablissement', String),
        ('dateDernierTraitementEtablissement',
         Datetime(time_unit='us', time_zone=None)),
        ('etablissementSiege', Boolean),
        ('nombrePeriodesEtablissement', Int64),
        ('complementAdresseEtablissement', String),
        ('numeroVoieEtablissement', String),
        ('indiceRepetitionEtablissement', String),
        ('dernierNumeroVoieEtablissement', String),
        ('indiceRepetitionDernierNumeroVoieEtablissement', String),
        ('typeVoieEtablissement', String),
        ('libelleVoieEtablissement', String),
        ('codePostalEtablissement', String),
        ('libelleCommuneEtablissement', String),
        ('libelleComm

In [6]:
df_eta.head()

siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,dernierNumeroVoieEtablissement,indiceRepetitionDernierNumeroVoieEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,identifiantAdresseEtablissement,coordonneeLambertAbscisseEtablissement,coordonneeLambertOrdonneeEtablissement,complementAdresse2Etablissement,numeroVoie2Etablissement,indiceRepetition2Etablissement,typeVoie2Etablissement,libelleVoie2Etablissement,codePostal2Etablissement,libelleCommune2Etablissement,libelleCommuneEtranger2Etablissement,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
str,i64,str,str,date,str,i64,str,datetime[μs],bool,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,str,str,str,str,str,str,str,str
"""000325175""",16,"""00032517500016""","""O""",2000-09-26,"""NN""",null,"""3212ZZ""",2024-03-22 15:40:57,false,3,null,null,null,null,null,null,"""MANIHI COTE MONTAGNE TUAMOTU""","""98770""","""MANIHI""",null,null,"""98727""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2009-05-27,"""F""",null,null,null,null,"""32.12Z""","""NAFRev2""","""N"""
"""000325175""",24,"""00032517500024""","""O""",2008-05-20,"""NN""",null,null,2025-01-27 02:35:48,false,2,null,"""1""",null,null,null,"""PLACE""","""LEONCE DE SEYNES""","""84000""","""AVIGNON""",null,null,"""84007""",null,null,null,null,"""84007cvsr_B""","""851150.0982592739""","""6317267.146094748""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,2011-10-21,"""F""","""TAHITI PERLES CREATIONS""",null,null,null,"""47.89Z""","""NAFRev2""","""N"""
"""000325175""",32,"""00032517500032""","""O""",2009-05-27,"""NN""",null,null,2025-01-27 02:31:40,false,2,"""ECONOMIS""","""6""",null,null,null,"""AVENUE""","""FRANCOIS MAURIAC""","""84000""","""AVIGNON""",null,null,"""84007""",null,null,null,null,"""840072225_B""","""848084.2366812509""","""6316548.347114814""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,2011-10-21,"""F""","""TAHITI PERLES CREATIONS""",null,null,null,"""32.12Z""","""NAFRev2""","""N"""
"""000325175""",40,"""00032517500040""","""O""",2011-10-21,"""NN""",null,"""3212ZZ""",2024-03-30 05:48:29,false,2,null,"""170""",null,null,null,"""AVENUE""","""DU COL DE L’ANGE""","""13420""","""GEMENOS""",null,null,"""13042""",null,null,null,null,"""130420096_B""","""913005.2782392878""","""6245869.653506171""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,2014-01-07,"""F""","""TAHITI PERLES CREATION""",null,null,null,"""32.12Z""","""NAFRev2""","""N"""
"""000325175""",57,"""00032517500057""","""O""",2014-01-07,"""NN""",null,null,2024-03-30 05:29:10,false,2,null,"""61""",null,null,null,"""RUE""","""MARX DORMOY""","""13004""","""MARSEILLE""",null,null,"""13204""",null,null,null,null,"""132045859_B""","""894589.9700529675""","""6247476.0260883765""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,2018-02-07,"""F""","""TAHITI PERLES CREATION""",null,null,null,"""32.12

# Stock Unite Legale

In [ ]:
import polars as pl
import pyarrow.parquet as pq
import sys

# Indique le chemin de ton fichier
filepath = "../Data/raw/StockUniteLegale_utf8.parquet" 

print("--- Début de la lecture 'bypass' ---")

try:
    # ÉTAPE 1 : On lit avec PyArrow, qui est robuste.
    print(f"Lecture du fichier via PyArrow : {filepath}")
    table_arrow = pq.read_table(
        filepath,
    )
    
    # ÉTAPE 2 : On passe la data à Polars sans la copier (gain de performance).
    print("Conversion de la table PyArrow en DataFrame Polars...")
    df_unit_legal = pl.from_arrow(table_arrow)
    
    print("--- SUCCÈS ! ---")
    print("Le DataFrame est maintenant dans Polars, prêt pour la transformation.")
    print(df_unit_legal.head())

except Exception as e:
    print(f"--- ERREUR ---", file=sys.stderr)
    print(f"Impossible de lire le fichier, même avec PyArrow : {e}", file=sys.stderr)

--- Début de la lecture 'bypass' ---
Lecture du fichier via PyArrow : ../Data/raw/StockUniteLegale_utf8.parquet
Conversion de la table PyArrow en DataFrame Polars...
--- SUCCÈS ! ---
Le DataFrame est maintenant dans Polars, prêt pour la transformation.
shape: (5, 34)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ siren     ┆ statutDif ┆ unitePurg ┆ dateCreat ┆ … ┆ nicSiegeU ┆ economieS ┆ societeMi ┆ caracter │
│ ---       ┆ fusionUni ┆ eeUniteLe ┆ ionUniteL ┆   ┆ niteLegal ┆ ocialeSol ┆ ssionUnit ┆ eEmploye │
│ str       ┆ teLegale  ┆ gale      ┆ egale     ┆   ┆ e         ┆ idaireUni ┆ eLegale   ┆ urUniteL │
│           ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ teL…      ┆ ---       ┆ egale    │
│           ┆ str       ┆ bool      ┆ date      ┆   ┆ i64       ┆ ---       ┆ str       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆ str       ┆           ┆ str      │
╞═══════════╪═══════════╪

In [9]:
df_unit_legal.describe()

statistic,siren,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,sigleUniteLegale,sexeUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,prenomUsuelUniteLegale,pseudonymeUniteLegale,identifiantAssociationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,dateDernierTraitementUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,dateDebut,etatAdministratifUniteLegale,nomUniteLegale,nomUsageUniteLegale,denominationUniteLegale,denominationUsuelle1UniteLegale,denominationUsuelle2UniteLegale,denominationUsuelle3UniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,nomenclatureActivitePrincipaleUniteLegale,nicSiegeUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,caractereEmployeurUniteLegale
str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,f64,str,f64,str,f64,str,str,str,str,str,str,str,str,f64,str,str,f64,str,str,str
"""count""","""28882409""","""28882409""",4.267752e6,"""27710121""","""3500030""","""16301667""","""16310994""","""7451768""","""4630285""","""2939334""","""16310994""","""2866012""","""1050584""","""28882409""",2.008421e6,"""28882409""",2.8882409e7,"""10546073""",1.0546073e7,"""28474772""","""28882409""","""16315955""","""5051773""","""15349351""","""4158591""","""2776349""","""2772761""",2.8882409e7,"""28860884""","""28860884""",2.8882409e7,"""6167001""","""831984""","""0"""
"""null_count""","""0""","""0""",2.4614657e7,"""1172288""","""25382379""","""12580742""","""12571415""","""21430641""","""24252124""","""25943075""","""12571415""","""26016397""","""27831825""","""0""",2.6873988e7,"""0""",0.0,"""18336336""",1.8336336e7,"""407637""","""0""","""12566454""","""23830636""","""13533058""","""24723818""","""26106060""","""26109648""",0.0,"""21525""","""21525""",0.0,"""22715408""","""28050425""","""28882409"""
"""mean""",null,null,1.0,"""2005-10-22 13:11:56.004134""",null,null,null,null,null,null,null,null,null,null,2023.0,"""2024-05-27 05:13:09.723577""",2.419245,null,2023.0,"""2013-08-19 06:51:08.445851""",null,null,null,null,null,null,null,3246.573881,null,null,19.358745,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,1.74063,null,0.0,null,null,null,null,null,null,null,null,2736.41353,null,null,43.78713,null,null,null
"""min""","""000325175""","""O""",1.0,"""0001-01-16""",""" ""","""F""",""" MILA """,""" """,""" """,""" """,""" MILA """,""" """,""" ""","""00""",2023.0,"""2024-03-22 14:26:06""",1.0,"""ETI""",2023.0,"""0001-01-16""","""A""",""" ACKAH-ENYIMAH""",""" """,""" 'ASSOCIATION DES COMBATTANT…",""" AU P'TIT ZINC ""","""& CO FRAIKIN & HOANG IMMOBILI…","""& ENSEIGNEMENTS …",1000.0,"""00.00""","""NAF1993""",1.0,"""N""","""N""",null
"""25%""",null,null,null,"""1995-01-19""",null,null,null,null,null,null,null,null,null,null,2023.0,"""2024-03-22 14:26:06""",1.0,null,2023.0,"""2008-01-01""",null,null,null,null,null,null,null,1000.0,null,null,13.0,null,null,null
"""50%""",null,null,null,"""2010-05-01""",null,null,null,null,null,null,null,null,null,null,2023.0,"""2024-03-22 14:26:06""",2.0,null,2023.0,"""2017-07-10""",null,null,null,null,null,null,null,1000.0,null,null,16.0,null,null,null
"""75%""",null,null,null,"""2019-11-02""",null,null,null,null,null,null,null,null,null,null,2023.0,"""2024-03-22 14:26:06""",3.0,null,2023.0,"""2022-09-15""",null,null,null,null,null,null,null,5599.0,null,null,21.0,null,null,null
"""max""","""999992357""","""P""",1.0,"""3023-01-06""","""€CO-MARKET""","""[ND]""","""[ND]""","""[ND]""","""[ND]""","""[ND]""","""[ND]""","""ØNYYX""","""w931024899""","""NN""",2023.0,"""2025-11-01 00:04:39""",71.0,"""PME""",2023.0,"""9998-11-10""","""C""","""[ND]""","""[ND]""","""€URO MULTI-SERVICES""","""¿A.COM PUBLICITE""","""[ND]""","""[ND]""",9970.0,"""99.0Z""","""NAP""",76665.0,"""O""","""O""",null


In [10]:
df_unit_legal.schema

Schema([('siren', String),
        ('statutDiffusionUniteLegale', String),
        ('unitePurgeeUniteLegale', Boolean),
        ('dateCreationUniteLegale', Date),
        ('sigleUniteLegale', String),
        ('sexeUniteLegale', String),
        ('prenom1UniteLegale', String),
        ('prenom2UniteLegale', String),
        ('prenom3UniteLegale', String),
        ('prenom4UniteLegale', String),
        ('prenomUsuelUniteLegale', String),
        ('pseudonymeUniteLegale', String),
        ('identifiantAssociationUniteLegale', String),
        ('trancheEffectifsUniteLegale', String),
        ('anneeEffectifsUniteLegale', Int64),
        ('dateDernierTraitementUniteLegale',
         Datetime(time_unit='us', time_zone=None)),
        ('nombrePeriodesUniteLegale', Int64),
        ('categorieEntreprise', String),
        ('anneeCategorieEntreprise', Int64),
        ('dateDebut', Date),
        ('etatAdministratifUniteLegale', String),
        ('nomUniteLegale', String),
        ('nomUsageUni

In [11]:
df_unit_legal.head()

siren,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,sigleUniteLegale,sexeUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,prenomUsuelUniteLegale,pseudonymeUniteLegale,identifiantAssociationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,dateDernierTraitementUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,dateDebut,etatAdministratifUniteLegale,nomUniteLegale,nomUsageUniteLegale,denominationUniteLegale,denominationUsuelle1UniteLegale,denominationUsuelle2UniteLegale,denominationUsuelle3UniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,nomenclatureActivitePrincipaleUniteLegale,nicSiegeUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,caractereEmployeurUniteLegale
str,str,bool,date,str,str,str,str,str,str,str,str,str,str,i64,datetime[μs],i64,str,i64,date,str,str,str,str,str,str,str,i64,str,str,i64,str,str,str
"""000325175""","""O""",null,2000-09-26,null,"""M""","""THIERRY""",null,null,null,"""THIERRY""",null,null,"""NN""",null,2024-03-22 14:26:06,6,"""PME""",2023,2018-02-07,"""A""","""JANOYER""",null,null,null,null,null,1000,"""32.12Z""","""NAFRev2""",65,null,null,null
"""001807254""","""O""",null,1972-05-01,null,"""M""","""JACQUES-LUCIEN""",null,null,null,"""JACQUES-LUCIEN""",null,null,"""NN""",null,2024-03-22 14:26:06,5,null,null,2014-12-31,"""C""","""BRETON""",null,null,null,null,null,1000,"""85.59A""","""NAFRev2""",22,null,null,null
"""005410220""","""O""",true,1954-12-25,null,"""M""","""GEORGES""",null,null,null,"""GEORGES""",null,null,"""NN""",null,2024-03-22 14:26:06,1,null,null,1988-03-31,"""C""","""WATTEBLED""",null,null,null,null,null,1000,"""22.02""","""NAP""",15,null,null,null
"""005410345""","""O""",true,null,null,"""M""","""MICHEL""",null,null,null,"""MICHEL""",null,null,"""NN""",null,2024-03-22 14:26:06,1,null,null,1984-12-25,"""C""","""DEBRAY""",null,null,null,null,null,1000,"""79.06""","""NAP""",10,null,null,null
"""005410394""","""O""",true,1954-12-25,null,"""M""","""ROBERT""","""ALFRED""",null,null,"""ROBERT""",null,null,"""NN""",null,2024-03-22 14:26:06,1,null,null,1987-12-01,"""C""","""DAULT""",null,null,null,null,null,1000,"""64.42""","""NAP""",18,null,null,null


# Detail Bilan

In [1]:
import polars as pl
import pyarrow.parquet as pq  # C'est le moteur 4x4
import sys

# Indique le chemin de ton fichier
filepath = "../Data/raw/ExportDetailBilan.parquet" 

print("--- Début de la lecture 'bypass' ---")

try:
    # ÉTAPE 1 : On lit avecPyArrow et on ne prend que les colonnes siren et liasse.
    print(f"Lecture du fichier via PyArrow (le moteur robuste) : {filepath}")
    table_arrow = pq.read_table(
        filepath,
        columns=["siren", "liasse"]
    )
    
    # ÉTAPE 2 : On passe la data à Polars sans la copier (gain de performance).
    print("Conversion de la table PyArrow en DataFrame Polars...")
    df_bilan = pl.from_arrow(table_arrow)
    
    print("--- SUCCÈS ! ---")
    print("Le DataFrame est maintenant dans Polars, prêt pour la transformation.")
    print(df_bilan.head())

except Exception as e:
    print(f"--- ERREUR ---", file=sys.stderr)
    print(f"Impossible de lire le fichier, même avec PyArrow : {e}", file=sys.stderr)

--- Début de la lecture 'bypass' ---
Lecture du fichier via PyArrow (le moteur robuste) : ../Data/raw/ExportDetailBilan.parquet
Conversion de la table PyArrow en DataFrame Polars...
--- SUCCÈS ! ---
Le DataFrame est maintenant dans Polars, prêt pour la transformation.
shape: (5, 2)
┌───────────┬─────────────────────────────────┐
│ siren     ┆ liasse                          │
│ ---       ┆ ---                             │
│ str       ┆ list[struct[2]]                 │
╞═══════════╪═════════════════════════════════╡
│ 005420120 ┆ [{"HF",111571}, {"BH-BI",2559}… │
│ 005420120 ┆ [{"GK",74833}, {"AS",90304}, …… │
│ 005420120 ┆ [{"HF",151562}, {"NH",5850813}… │
│ 005420120 ┆ [{"HC",918483}, {"CU-CV",66917… │
│ 005420120 ┆ [{"AO",425420}, {"CH-CI",14383… │
└───────────┴─────────────────────────────────┘


In [2]:
df_bilan.describe()


statistic,siren,liasse
str,str,f64
"""count""","""4976444""",4.976444e6
"""null_count""","""0""",0.0
"""mean""",null,null
"""std""",null,null
"""min""","""005420120""",null
"""25%""",null,null
"""50%""",null,null
"""75%""",null,null
"""max""","""999990542""",null


In [4]:
df_bilan.schema

Schema([('siren', String),
        ('liasse', List(Struct({'key': String, 'value': Int32})))])

In [3]:
df_bilan.head()

siren,liasse
str,list[struct[2]]
"""005420120""","[{""HF"",111571}, {""BH-BI"",2559}, … {""7C"",220031}]"
"""005420120""","[{""GK"",74833}, {""AS"",90304}, … {""BV"",3518}]"
"""005420120""","[{""HF"",151562}, {""NH"",5850813}, … {""DQ"",2967321}]"
"""005420120""","[{""HC"",918483}, {""CU-CV"",66917249}, … {""FJ"",104225}]"
"""005420120""","[{""AO"",425420}, {""CH-CI"",14383}, … {""FR"",520363}]"


In [6]:
# affcihe toute la colonee liasse du la premier lig e
df_bilan["liasse"][0]



""
struct[2]
"{""HF"",111571}"
"{""BH-BI"",2559}"
"{""CJ-CK"",15117606}"
"{""DO"",0}"
"{""EA"",123502}"
…
"{""FR"",470896}"
"{""I4"",0}"
"{""ØH"",0}"
